In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle
import json
from os.path import expanduser

In [2]:
experiment_name = 'P3856'
run_name = 'P3856_YHE211_1_Slot1-1_1_5104'

In [3]:
BASE_MAXQUANT_DIR = '{}/MQ-analysis-of-P3856/combined'.format(expanduser('~'))
SCANS_FILE = '{}/txt/pasefMsmsScans.txt'.format(BASE_MAXQUANT_DIR)
ALLPEPTIDES_FILE = '{}/txt/allPeptides.txt'.format(BASE_MAXQUANT_DIR)

In [4]:
scans_df = pd.read_csv(SCANS_FILE, sep='\t')
scans_df = scans_df[(scans_df['Raw file'] == run_name)]

In [5]:
allpeptides_df = pd.read_csv(ALLPEPTIDES_FILE, sep='\t')
allpeptides_df = allpeptides_df[(allpeptides_df['Raw file'] == run_name)]
allpeptides_df = allpeptides_df[allpeptides_df['Intensity'].notnull() & allpeptides_df['Pasef MS/MS IDs'].notnull()]

In [6]:
# convert the ;-separated list of ids to a Python list
allpeptides_df['pasef_msms_ids_list'] = allpeptides_df['Pasef MS/MS IDs'].str.split(";").apply(lambda x: [int(i) for i in x])

#### find the selected precursor

In [7]:
precursor_id = 20114

In [8]:
scans_df[scans_df['Precursor'] == precursor_id]

,Raw file,Frame,Precursor,IsolationMz,IsolationWidth,ScanNumBegin,ScanNumEnd,CollisionEnergy,Index
2139380,P3856_YHE211_1_Slot1-1_1_5104,16364,20114,780.486511,2.804865,590,615,37.22656,44678
2139394,P3856_YHE211_1_Slot1-1_1_5104,16365,20114,780.486511,2.804865,590,615,37.22656,44692
2139407,P3856_YHE211_1_Slot1-1_1_5104,16366,20114,780.486511,2.804865,590,615,37.22656,44705
2139422,P3856_YHE211_1_Slot1-1_1_5104,16368,20114,780.486511,2.804865,590,615,37.22656,44720


In [9]:
pasefMsmsScans_indexes_for_precursor = scans_df[scans_df['Precursor'] == precursor_id].Index.to_list()
pasefMsmsScans_indexes_for_precursor

[44678, 44692, 44705, 44720]

#### find the entry in allPeptides for the precursor

In [10]:
ap_df = allpeptides_df[allpeptides_df.pasef_msms_ids_list.apply(lambda x: len(set(pasefMsmsScans_indexes_for_precursor) & set(x)) > 0)]
ap_df

,Raw file,Charge,m/z,Mass,Number of data points,Number of frames,Number of isotopic peaks,Isotope correlation,Mass fractional part,Mass deficit,...,Max frame index,Ion mobility index,Ion mobility index length,Ion mobility index length (FWHM),Intensity,Intensities,Number of pasef MS/MS,Pasef MS/MS IDs,MS/MS scan number,pasef_msms_ids_list
1192099,P3856_YHE211_1_Slot1-1_1_5104,3,780.02997,2337.0681,4190,11,5,0.987452,0.068079,-0.046973,...,7217,534,141,27,169890.0,NaN,3,44491;44505;44720,20432.0,"[44491, 44505, 44720]"


In [11]:
apl_index = int(ap_df['MS/MS scan number'].iloc[0])
apl_index

20432